# Recommendations Engine For Movies

The moving rating data used in this excercise was taken from [Grouplens](https://grouplens.org/datasets/movielens/).

## Importing Data

In [1]:
import pandas as pd
import numpy as np
import os
import re
from collections import Counter

In [2]:
base_dir = "./cornell-data/scale_whole_review/scale_whole_review/{}/txt.parag"

In [93]:
class DataLoader:
    base_dir = "./cornell-data/scale_whole_review/scale_whole_review/{}/txt.parag"
    review_titles = {}
    
    def __init__(self, name):
        self.name = name
        if name == "Dennis+Schwartz":
            self.title_setter = self.title_setter_by_director_token
        else:
            self.title_setter = self.title_setter_by_frequence
    
    def filling_reviews_titles(self):
        directory = self.base_dir.format(self.name)
        #Selecting an title setter:
        review_files = os.listdir(directory)
        for review_file in review_files:
            review_text = open_file(directory,review_file)
            review_id = re.split("\.",review_file)[0]
            s = self.title_setter(review_text, review_id)
            
    def open_file(self,directory,review_file):
        with open("{}/{}".format(directory,review_file), "rb") as file:
            text_review = file.read().decode('utf-8',errors='ignore')
        return text_review
    
    def title_setter_by_director_token(self,review_text, review_id): 
        try:
            title = re.findall('([A-Z,.\:\-\!\/\'\s\d]+)\s+.*\([dD]irect', review_text)
            self.review_titles[review_id] = re.split("\,",title[0])[0]
        except:
            pass

    def title_setter_by_frequence(self,review_text, review_id):
        try:
            words_array = re.findall('([A-Z][A-Z\:\-\!\/\'\s\d]+) [a-z]',review_text)
            title = Counter(words_array).most_common(1)[0][0]
            self.review_titles[review_id] = title
        except:
            pass

In [97]:
for name in ["Dennis+Schwartz","James+Berardinelli","Scott+Renshaw","Steve+Rhodes"]:
    data_loader = DataLoader(name)
    data_loader.filling_reviews_titles()

In [98]:
data_loader.review_titles

{'28454': 'THINGS CHANGE',
 '23229': 'CABE AND MRS. MILLER',
 '25246': 'PERFECT STORM',
 '21031': 'PLEASANTVILLE',
 '23855': 'NAKED KISS',
 '28018': 'TRIP',
 '24274': 'NO WAY TO TREAT A LADY',
 '29852': 'SERENDIPITY',
 '28798': 'CODE UNKNOWN',
 '29429': 'PLUNDER ROAD',
 '25583': 'WITNESS TO MURDER',
 '29517': 'RED PLANET MARS',
 '28394': 'VALLEY OF THE DOLLS',
 '23532': 'NAME OF THE ROSE',
 '29757': 'RATCATCHER',
 '27319': 'CAST AWAY',
 '20422': 'NO END',
 '25909': 'LIFE AND TIMES OF HANK GREENBERG',
 '25579': 'ROSETTA',
 '29519': 'CURSE OF THE JADE SCORPION',
 '28420': 'IN THE MOOD FOR LOVE',
 '19745': 'BESIEGED',
 '26721': 'SHE-CREATURE',
 '20311': 'RED SQUIRREL',
 '22019': 'DAY AT THE BEACH',
 '29703': 'WHERE THE MONEY IS',
 '21910': 'BLOOD OF A POET',
 '23518': 'ILLUMINATA',
 '28602': 'INTIMATE RELATIONS',
 '17118': 'LES BICHES',
 '27047': 'GIRL ON THE BRIDGE',
 '25205': 'LAST NIGHT',
 '26924': 'HALLELUJAH THE HILLS',
 '27839': 'SHOWER',
 '29347': 'OUTSIDE MAN',
 '19979': 'SEARCHER

# ratings_df = pd.read_csv("./ml-latest-small/ratings.csv")
movies_df = pd.read_csv("./ml-latest-small/movies.csv")

In [100]:
movies_df.head(2)

NameError: name 'movies_df' is not defined

In [138]:
ratings_df.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


## Creating the User Item Table

To create the User - Item Matrix lets use a pivot table.

In [139]:
R_df = ratings_df.pivot(index="userId", columns="movieId", values="rating").fillna(0)
R_df.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Singular Value Decomposition

In [193]:
R = R_df.as_matrix()
from scipy.sparse.linalg import svds
U,sigma,Vt = svds(R,k=np.min([(np.min(R.shape)-1),8]))

## Making Predictions

In [194]:
all_users_predicted_ratings = np.dot(np.dot(U,np.diag(sigma)),Vt)

In [196]:
all_users_predicted_ratings

array([[  1.16508645e-02,  -3.69107499e-03,   1.51857682e-02, ...,
         -3.16552788e-04,  -1.89931673e-04,   1.50938488e-03],
       [  1.64924370e+00,   1.55426966e+00,   6.10894118e-01, ...,
          1.56953603e-03,   9.41721616e-04,   6.51877834e-02],
       [  1.03236874e+00,   4.27261247e-01,   1.05487502e-01, ...,
          1.46437781e-03,   8.78626683e-04,  -3.43021502e-02],
       ..., 
       [  2.09072274e-01,   4.47633373e-02,   3.46432779e-02, ...,
         -4.41212343e-04,  -2.64727406e-04,  -2.78438712e-02],
       [  7.48372680e-01,   3.42115840e-01,   1.21867871e-01, ...,
          1.70906435e-04,   1.02543861e-04,  -1.84469764e-02],
       [  2.26192041e+00,   3.95873315e-01,   4.23334198e-02, ...,
          4.44739385e-03,   2.66843631e-03,  -1.13711249e-01]])

## How to execute it

The below command executes the movies recommendation engine for the user with id 2, it making 6 recommendations, and showing the top 8 historical ratings.
```bash
python recommender.py '{"user id":2, "Recommendation limit": 3, "Historical limit":8}'
```

## References

1. [Generals on movies recommendation systems.](https://blog.statsbot.co/recommendation-system-algorithms-ba67f39ac9a3)
1. [Matrix factorization recommender.](https://beckernick.github.io/matrix-factorization-recommender/)
2. [A movie recommendation system inplemented on Spark.](https://www.packtpub.com/books/content/building-recommendation-engine-spark)
3. [About the Netflix recommendation system.](https://medium.com/netflix-techblog/netflix-recommendations-beyond-the-5-stars-part-1-55838468f429)
4. [Performance metrics.](https://en.wikipedia.org/wiki/Information_retrieval#Precision_at_K)
5. [Movie ratings dataset.](https://grouplens.org/datasets/movielens/)